In [2]:
from openpyxl import load_workbook
import re
wb = load_workbook("TM4C_controller_pin_specs.xlsx")
print(wb)
ws = wb.active


In [5]:
for row in ws.iter_rows(min_row=9, max_col=6, max_row=136):
    print (row[0].value,row[1].value,row[5].value)

1 AIN15 VCC_M1V8, direct measure of always-on 1.8v maintenance supply
2 AIN14 VCC_M3V3,  divided-by-2 measure of always-on 3.3v maintenance supply
3 AIN13 VCC_2V5,  divided-by-2 measure of 2.5v legacy timing supply.
4 AIN12 VCC_12V,  divided-by-6 measure of always-on 12v main input supply.
5 PQ0 TM4C_TO_VU7P_0: This output to the VU7P is currently undefined
6 PQ1 TM4C_FROM_VU7P_0: This input from the VU7P is currently undefined
7 VDD VCC_M3V3: 3.3 volts from the management regulator. It is present whenever the board has 12v supplied.
8 VDDA VCC_M3V3
9 VREFA+ VREF_2.5V: This 2.5v voltage sets the value that will correspond to the ADC full scale valu of 4095 (0xfff)
10 GNDA GND
11 PQ2 TM4C_DIP_SW_2: Wired to position #4 of switch SW5, the VU7P DIP SWITCH. A HI level means that the VU7P should be powered; a LO level means that the VU7P should be left unpowered. The pullup on the switch is energized from the VCC_M3V3 source, so it is valid whenever the board has 12V supplied. 
12 AIN0 V_MG

In [13]:
class gpio_pin_t:
    name = ""
    pin = -1
    port = ""
    local_pin = -1 
    is_input = True
    def __init__(self, name, pin, port, local_pin, is_input):
        self.pin = pin
        self.name = name
        self.port = port
        self.local_pin = local_pin
        self.is_input = is_input
        
class pin_t:
    name = ""
    pin = -1
    def __init__(self, name, pin) :
        self.pin = pin
        self.name = name


In [16]:
pinsl=[]
allpinsl = []

for row in ws.iter_rows(min_row=9, max_col=6, max_row=136):
    
    pin = row[0].value
    is_input = True
    if ( row[2].value == "O"):
        is_input = False
 
    #if ( re.search("nput", row[5].value) == [] ) :
    #    is_input = False
    signame=re.split(", |: ",row[5].value)[0]
    signame = re.sub('/', '_', signame)
    #print  row[4].value, re.split(r'GPIO port +([A-Z]+) bit ([0-9]+)', row[4].value)
    #print pin, signame, gpio_port, gpio_pin
    #print "#define ", signame, pin, "// Port, local pin=", gpio_port, gpio_pin, ", input=", is_input
    is_not_gpio = row[4].value.find("GPIO")
    allpinsl.append(pin_t(name=signame, pin=pin))
    if ( is_not_gpio ):
        #print "not gpio ->",row[4].value, row[5].value
        continue
    [undef, gpio_port, gpio_pin, undef ]  = re.split(r'GPIO port +([A-Z]+) bit ([0-9]+)', row[4].value)
    pinsl.append(gpio_pin_t(name=signame,pin=pin, port=gpio_port, local_pin = gpio_pin, is_input=is_input))

#print len(pinsl)
for p in pinsl:
    print( "#define ", p.name, p.pin, "// port, local_pin =", p.port, p.local_pin)

print( "\n\n\n")
print( "switch(pin):")
print( "{")
for p in pinsl:
    print( "\tcase ", p.name, ":\n\t{")
    print("\t\tgpio_port = GPIO_PORT%s_BASE;" % p.port)
    print("\t\tgpio_pin  = GPIO_PIN_%s;" % p.local_pin)
    print("\t\tbreak;\n\t}")
print("\tdefault:")
print("\t\tgpio_port = -1;")
print( "\t\tgpio_pin  = -1;")
print( "\t\tbreak;")
print( "}"    )

#define  TM4C_TO_VU7P_0 5 // port, local_pin = Q 0
#define  TM4C_FROM_VU7P_0 6 // port, local_pin = Q 1
#define  TM4C_DIP_SW_2 11 // port, local_pin = Q 2
#define  V_MGTY1_AVTT_OK 22 // port, local_pin = C 7
#define  V_MGTY1_AVCC_OK 23 // port, local_pin = C 6
#define  V_MGTY2_AVCC_OK 24 // port, local_pin = C 5
#define  V_MGTY2_AVTT_OK 25 // port, local_pin = C 4
#define  V_FPGA_PROGRAM 27 // port, local_pin = Q 3
#define  VCC_1V8_PG 29 // port, local_pin = H 0
#define  VCC_3V3_PG 30 // port, local_pin = H 1
#define  V_VCCINT_PG_B 31 // port, local_pin = H 2
#define  V_VCCINT_PG_A 32 // port, local_pin = H 3
#define  CTRL_V_VCCINT_PWR_EN 37 // port, local_pin = A 4
#define  CTRL_VCC_1V8_PWR_EN 38 // port, local_pin = A 5
#define  _FPGA_I2C_RESET 42 // port, local_pin = F 0
#define  CTRL_VCC_3V3_PWR_EN 43 // port, local_pin = F 1
#define  CTRL_K_VCCINT_PWR_EN 44 // port, local_pin = F 2
#define  _PWR_I2C_RESET 45 // port, local_pin = F 3
#define  TM4C_TP1 46 // port, local_pin = F 4
#d

In [11]:
print( "const char* const pin_names[] ="    )
print( "{")
for p in allpinsl:
    print( "\"%s\",// %d "% (p.name, p.pin) )
print("};")    

const char* const pin_names[] =
{
"VCC_M1V8",// 1 
"VCC_M3V3",// 2 
"VCC_2V5",// 3 
"VCC_12V",// 4 
"TM4C_TO_VU7P_0",// 5 
"TM4C_FROM_VU7P_0",// 6 
"VCC_M3V3",// 7 
"VCC_M3V3",// 8 
"VREF_2.5V",// 9 
"GND",// 10 
"TM4C_DIP_SW_2",// 11 
"V_MGTY1_AVTT",// 12 
"V_MGTY1_AVCC",// 13 
"V_MGTY1_VCCAUX",// 14 
"V_VCCINT",// 15 
"VCC_M3V3",// 16 
"GND",// 17 
"VCC_3V3",// 18 
"V_MGTY2_VCCAUX",// 19 
"V_MGTY2_AVCC",// 20 
"V_MGTY2_AVTT",// 21 
"V_MGTY1_AVTT_OK",// 22 
"V_MGTY1_AVCC_OK",// 23 
"V_MGTY2_AVCC_OK",// 24 
"V_MGTY2_AVTT_OK",// 25 
"VCC_M3V3",// 26 
"V_FPGA_PROGRAM",// 27 
"VCC_M3V3",// 28 
"VCC_1V8_PG",// 29 
"VCC_3V3_PG",// 30 
"V_VCCINT_PG_B",// 31 
"V_VCCINT_PG_A",// 32 
"BLADE_IPMC_TX",// 33 
"BLADE_IPMC_RX",// 34 
"FRONT_PANEL_UART_RX",// 35 
"FRONT_PANEL_UART_TX",// 36 
"CTRL_V_VCCINT_PWR_EN",// 37 
"CTRL_VCC_1V8_PWR_EN",// 38 
"VCC_M3V3",// 39 
"FPGA_I2C_SCL",// 40 
"FPGA_I2C_SDA",// 41 
"_FPGA_I2C_RESET",// 42 
"CTRL_VCC_3V3_PWR_EN",// 43 
"CTRL_K_VCCINT_PWR_EN",// 44 
"_PWR_I

In [15]:
f = open("/tmp/test_rev1.def", "w")

# x-macros version
for p in pinsl:
    direction = "true" # c-style boolean values
    if ( p.is_input == False ):
        direction = "false"
    f.write(f'   X({p.name}, {p.pin}, {p.port}, {p.local_pin}, {direction}) \n')
f.write("#undef X\n")
f.close()